In [1]:
from blocksnet.machine_learning.strategy.xgboost import XGBoostClassificationStrategy
from blocksnet.machine_learning.strategy.catboost import CatBoostClassificationStrategy

RANDOM_STATE = 42
N_JOBS = -1

xgb = XGBoostClassificationStrategy(model_params={
    "n_estimators": 200,
    "max_depth": 7,
    "learning_rate": 0.05,
    "scale_pos_weight": 1,  # Аналог class_weight для XGBoost
    'n_jobs': N_JOBS,
    'random_state': RANDOM_STATE
})

cb = CatBoostClassificationStrategy(model_params={
    "iterations": 200,  # CatBoost использует iterations вместо n_estimators
    "depth": 7,
    "learning_rate": 0.05,
    "thread_count": N_JOBS, # Аналог n_jobs для CatBoost
    "auto_class_weights": "Balanced",
    "random_seed": RANDOM_STATE,
})

In [3]:
from blocksnet.machine_learning.strategy.sklearn.ensemble.voting.classification_strategy import SKLearnClassificationStrategy

SKLearnClassificationStrategy(strategies=[xgb, cb])

TypeError: Can't instantiate abstract class SKLearnClassificationStrategy with abstract methods predict, predict_proba, train